# Inspect Data of Employment Website 104

In [3]:
import pandas as pd
import gensim
import nltk
nltk.download('wordnet')

import jieba
from sklearn.manifold import TSNE

from bokeh.plotting import figure, show, output_notebook, save
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource
output_notebook()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Loading BokehJS ...

# Read Data

In [11]:
filename_dataset = './datasets/jobs_104.json'

dataset_df = pd.read_json(filename_dataset)

In [12]:
dataset_df.shape

(996, 8)

In [13]:
dataset_df.head()

,address,benefit,company,experience,requirement,requirement_others,salary,title
0,台北市內湖...,福利健全 ■ 優渥的薪資、獎金 ◎ 端午節、中秋節、年終獎金。 ◎ 定期依工作績效調...,芯鼎科技股份有限公司,不拘,- Driver development and fine tune of customiz...,"- A BS degree in Computer Engineering, Electri...",待遇面議 ...,系統應用工程師-台北(CD2320) ...
1,台北市大安...,＊獎金福利＊1. 年節禮金2. 生日禮金3. 年終分紅4. 人才介紹獎金＊休假福利＊1. ...,銓鴻資訊有限公司,1年以上,1. 研究市場數據和交易數據，並進行交易策略研發。 2. 開發模型和工具，監控分析交易程序...,將先以電子履歷篩選增加效率，請注意以下事項： ■ 履歷請詳述相關經驗並請提供 Github/...,"年薪 1,000,000~2,500,000元 ...",計量交易員 Quantitative Tr...
2,新北市新店...,《薪酬福利＠HTC》我們極力落實利潤與員工分享的精神，透過多元的薪酬組合與完善的福利方案，...,宏達電 HTC Corporation_宏達國際電子股份有限公司,不拘,Join the creative thinkers at HTC Healthcare. ...,Requirement:-MS or PhD in computer science or ...,待遇面議 ...,(RD S/W) DeepQ - Comp...
3,桃園市中壢...,正職/全職人員之福利與訓練制度： 【薪酬福利】 -每年依市場薪資水準評估薪資調整專案 -...,OK超商_來來超商股份有限公司,不拘,OK超商認為，用心經營的企業，首重人才培訓。 因此，我們投入了大量的訓練資源，積極培育新一...,1.具服務熱忱，科系、經驗不拘，須輪班。 2.須自備交通工具及電腦網路設備(透過e-Lear...,"月薪 31,000~33,000元 ...",儲備幹部(大桃園地區) ...
4,新竹縣竹北...,【工作福利】彈性上下班。 【休假福利】週休二日、國定假日、婚假、產假、陪產假、特休假、彈...,神亞科技股份有限公司,3年以上,影像相關之深度學習演算法研究與開發,1. 有開發image processing相關應用經驗 2. 熟悉深度學習相關算法（e.g...,待遇面議 ...,AI 工程師 ...


In [14]:
requirements = dataset_df['requirement'] + ' ' + dataset_df['requirement_others']
requirements[:2]

0    - Driver development and fine tune of customiz...
1    1. 研究市場數據和交易數據，並進行交易策略研發。  2. 開發模型和工具，監控分析交易程序...
dtype: object

In [15]:
requirements[0]

'- Driver development and fine tune of customized HW accelerator for OpenCV/OpenCL/OpenVX/CNN.- iCatchOS(RTOS) vision process framework development and maintain (SDK/BSP)- Integration and maintain of in-house design algorithms and performance fine tune.- Discuss/Design & co-work with customers for vision related requests. - A BS degree in Computer Engineering, Electrical Engineering, or Computer Science- At least 1-3 years of embedded software development experience (Senior – more than 5 years) - Familiar with C/C++ programming language, nice to have experience in Assembly language- Knowledge of Real-Time concepts and have RTOS experience (ThreadX , FreeRTOS, eCos, RTLinux, …)- Good understanding of CPU (e.g. ARM, MIPS ..)  architecture- Image processing & analysis algorithm, Machine Learning, Deep Learning framework development experience, e.g. CNN, OpenCL, OpenVX, OpenCV, OpenGL- Self-motivated, good attitude, skilled in meeting deadlines and work collaboratively in a team environmen

In [16]:
requirements[1]

'1. 研究市場數據和交易數據，並進行交易策略研發。  2. 開發模型和工具，監控分析交易程序運行情況研究，實施統計套利策略，並研發統計套利回測工具。  3. 熟悉python R 或者c++,有自主研發量化策略的能力。 4. 具備機率、統計、機器學習和金融市場研究經驗 。  求才若渴 如果你(妳)有下列特質：  是個懶人： 希望以最小的力氣達成目標，甚至提升品質，懂得善用工具增加生產力，重構提煉自有的程式庫。  有團隊精神： 懂得協同合作提升整體效益，將團隊目標置於個人之前。  充滿好奇心： 喜歡接觸、學習新事物，發現各種創新的可能。  熱愛程式： 充滿熱情想改變世界，將巧思及創造力注入每一行程式碼。   歡迎加入我們！ 一起邁向偉大航道 : ) 將先以電子履歷篩選增加效率，請注意以下事項： ■ 履歷請詳述相關經驗並請提供 Github/Sample Code/作品，加速審核 。 ■ 我們非常重視求職者的職務申請，履歷將由相關團隊主管嚴謹評估，因履歷眾多，意者我們將主動聯繫，請求職者悉知並耐心等候。'

# Preprocess
- Add custom Chinese phrases
- Tokenize Chinese/English
    - For English: Normalize, Lemmatize, Remove English Stopwords
    - For Chinese: Remove Chinese Stopwords,